#  Importing and Pre-processing the EEG dataset

In [ ]:
# Import necessary libraries
import os
from glob import glob
import mne
import numpy as np


files=glob('dataverse_files/*.edf')

control_paths=[j for j in files if  'h' in j.split('/')[1]]
patient_paths=[j for j in files if  's' in j.split('/')[1]]

# Define a function to read an EDF file, apply a bandpass filter, and epoch the data
def read_file(file):
    # Read the EDF file and load the data into memory
    data = mne.io.read_raw_edf(file, preload=True)
    # Set the EEG reference and apply a .1 to 45 Hz Hamming bandpass filter
    data.set_eeg_reference().filter(l_freq=.1, h_freq=45)
    # Create epochs of 20 seconds with no overlap and get the data
    data_epoched = mne.make_fixed_length_epochs(data, duration=20, overlap=0).get_data()
    return data_epoched

# Read and process the control and patient files
epoched_array_controls = [read_file(j) for j in control_paths]
epoched_array_patients = [read_file(j) for j in patient_paths]

# Create labels for the control and patient data (0 for controls, 1 for patients)
control_labels = [len(i)*[0] for i in epoched_array_controls]
patient_labels = [len(i)*[1] for i in epoched_array_patients]

# Combine the control and patient data and labels
data = epoched_array_controls + epoched_array_patients
labels = control_labels + patient_labels

# Create a list of subject identifiers
subjects = [i for i, j in enumerate(data) for _ in j]

# Convert the data, labels, and subjects to numpy arrays
data_array = np.vstack(data)
label_array = np.hstack(labels)
subject_array = np.array(subjects)


In [30]:

print(data_array.shape)

print(label_array.shape)

print(subject_array.shape)

(1433, 19, 5000)
(1433,)
(1433,)


# EEG Visualization 1:  Power Spectral Density and Topographical EEG Activity Mapping Across different frequency bands for one Scz subject and one healthy control

In [ ]:
#EEG Visualization 1: Power Spectral Density and Topographical EEG Activity Mapping Across different frequency bands
import matplotlib.pyplot as plt


raw_h = mne.io.read_raw_edf("dataverse_files/h01.edf", preload=True)
raw_s = mne.io.read_raw_edf("dataverse_files/s01.edf", preload=True)

#Unfiltered signal for the  healthy subject
raw_h.plot(start=300,duration=2, block=True, show=False, color='blue');
raw_s.plot(start=300,duration=2, block=True, show=False, color='red');

#Unfiltered signal for the schizophrenia patient
raw_s.plot(start=300,duration=2, block=True, show=False, color='red');

#filtered signal for the healthy subject
xfil_h=raw_h.filter(0.1, 45., fir_design='firwin')#Bandpass filtering
xfil_h.plot(start=300, duration=2, bgcolor='w',show=False, events=None,proj=False,color='green');#plotting of the filtered signal

#filtered signal for the schizophrenia patient
xfil_s=raw_s.filter(0.1, 45., fir_design='firwin')#Bandpass filtering
xfil_s.plot(start=300, duration=2, bgcolor='w',show=False, events=None,proj=False,color='red');#plotting of the filtered signal

#filtered psd for healthy subject
xfil_h.plot_psd(area_mode='range', tmax=100.0, show=False, average=True, color='green');
plt.title('Healthy Subject Filtered PSD Plot')

#filtered psd for schizophrenia patient
xfil_s.plot_psd(area_mode='range', tmax=100.0, show=False, average=True, color='red');
plt.title('Schizophrenia Patient Filtered PSD Plot')
 
montage = mne.channels.make_standard_montage('standard_1020')
raw_s.set_montage(montage)
raw_h.set_montage(montage)
#unfiltered psd for schizophrenia patient
spectrum_s = raw_s.compute_psd();
spectrum_s.plot(average=True, picks="data", exclude="bads", amplitude=False)
plt.title('Schizophrenia Patient Unfiltered PSD Plot');
#unfiltered psd for healthy subject
spectrum_h = raw_h.compute_psd();
spectrum_h.plot(average=True, picks="data", exclude="bads", amplitude=False)
plt.title('Healthy Subject Unfiltered PSD Plot');


#unfilted topomap for healthy subject
raw_h.plot_psd_topomap()
#unfiltered topomap for schizophrenia patient and healthy subject
raw_s.plot_psd_topomap()



# Plot the sensor locations for all the subjects with channel names(same for all)
mne.viz.plot_sensors(raw_h.info, show_names=True);
plt.show()


# EEG Visualization 2: EEG signals averaged ver epochs for all healthy controls and all schizophrenia patients

In [ ]:


#EEG Brain Wave Visualization
average_eeg_normal_controls=np.mean(data_array[label_array==0],axis=0)
average_eeg_schizophrenia=np.mean(data_array[label_array==1],axis=0)
import matplotlib.pyplot as plt

# List of channel names
channel_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']

# Create a new figure
fig, axs = plt.subplots(average_eeg_normal_controls.shape[0], 2, figsize=(20, 30))
fs=250
t=np.arange(0,20,1/fs)
# Plot the average EEG signal for each channel for normal controls
for i in range(average_eeg_normal_controls.shape[0]):
    axs[i, 0].plot(t,average_eeg_normal_controls[i])
    axs[i, 0].set_title('Channel ' + channel_names[i]+'(Normal)')
    axs[i, 0].set_xlabel('Time (s)')
    axs[i,0].set_ylabel('Amplitude (uV)')

# Plot the average EEG signal for each channel for schizophrenia
for i in range(average_eeg_schizophrenia.shape[0]):
    axs[i, 1].plot(t,average_eeg_schizophrenia[i])
    axs[i, 1].set_title('Channel ' + channel_names[i]+'(Schizophrenia)')
    axs[i, 1].set_xlabel('Time (s)')
    axs[i,1].set_ylabel('Amplitude (uV)')

# Show the plot
fig.suptitle('EEG signals averaged over epochs for normal controls and schizophrenia patients\n\n')
plt.tight_layout()

plt.show()


# Feature Extraction 1: Statistical Features 

In [6]:

from scipy import stats
def statistical_features(data):
    mean=np.mean(data,axis=-1)
    ptp=np.ptp(data,axis=-1)
    var=np.var(data,axis=-1)
    rms=np.sqrt(np.mean(data**2,axis=-1))  
    absolute_difference=np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)
    skewness=stats.skew(data,axis=-1)
    kurtosis=stats.kurtosis(data,axis=-1)
    return np.concatenate((mean,ptp,var,rms,absolute_difference,skewness,kurtosis),axis=-1)

features=[]
for data in data_array:
    features.append(statistical_features(data))
features=np.array(features)

In [45]:
print(features.shape)

(1433, 133)


# Classification with Logistic Regression using Statistical Features 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Initialize lists to store metrics
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
specificity_list = []



LR = LogisticRegression()
group_kf = GroupKFold(n_splits=5)
grid = {
    'classifier__C': [0.001,0.01,0.05,0.1,0.5, 1,3,5,7,10], 
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'], 
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
pipe = Pipeline([('scaler',StandardScaler()),('classifier',LR)])
gscv = GridSearchCV(pipe, grid, cv=group_kf, n_jobs=16, scoring='accuracy')

# Cross-validation loop
for train_index, val_index in group_kf.split(features, label_array, groups=subject_array):
    # Split data
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = label_array[train_index], label_array[val_index]

    # Fit and predict
    gscv.fit(X_train, y_train, groups=subject_array[train_index])
    y_pred = gscv.predict(X_val)

    # Calculate metrics
    accuracy_list.append(accuracy_score(y_val, y_pred))
    precision_list.append(precision_score(y_val, y_pred, average='weighted'))
    recall_list.append(recall_score(y_val, y_pred, average='weighted'))
    f1_list.append(f1_score(y_val, y_pred, average='weighted'))

    # Calculate specificity
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    specificity_list.append(tn / (tn+fp))

In [61]:
# Print average metrics
print('Average Accuracy : ', np.mean(accuracy_list))
print('Average Precision : ', np.mean(precision_list))
print('Average Sensitivity : ', np.mean(recall_list))
print('Average F1 Score : ', np.mean(f1_list))
print('Average Specificity : ', np.mean(specificity_list))
print('best parameters: ',gscv.best_params_)

Average Accuracy :  0.6582007936495025
Average Precision :  0.6792376260649167
Average Sensitivity :  0.6582007936495025
Average F1 Score :  0.6613312655174791
Average Specificity :  0.6496237528441366
best parameters:  {'classifier__C': 10, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}


# Classification with Random Forest Using Statistical Features

In [ ]:
#Random Forest Classifier Using Statistical Features

from sklearn.ensemble import RandomForestClassifier

# Define your classifier
RF = RandomForestClassifier()

# Define your grid
grid = {
    'classifier__n_estimators': [10, 50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['auto', 'sqrt']
}

# Define your pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', RF)])

# Define your grid search
gscv = GridSearchCV(pipe, grid, cv=group_kf, n_jobs=16, scoring='accuracy')

for train_index, val_index in group_kf.split(features, label_array, groups=subject_array):
    # Split data
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = label_array[train_index], label_array[val_index]

    # Fit and predict
    gscv.fit(X_train, y_train, groups=subject_array[train_index])
    y_pred = gscv.predict(X_val)

    # Calculate metrics
    accuracy_list.append(accuracy_score(y_val, y_pred))
    precision_list.append(precision_score(y_val, y_pred, average='weighted'))
    recall_list.append(recall_score(y_val, y_pred, average='weighted'))
    f1_list.append(f1_score(y_val, y_pred, average='weighted'))

    # Calculate specificity
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    specificity_list.append(tn / (tn+fp))



In [63]:
# Print average metrics
print('Average Accuracy : ', np.mean(accuracy_list))
print('Average Precision : ', np.mean(precision_list))
print('Average Sensitivity : ', np.mean(recall_list))
print('Average F1 Score : ', np.mean(f1_list))
print('Average Specificity : ', np.mean(specificity_list))
print('best parameters: ',gscv.best_params_)

Average Accuracy :  0.6376956133013285
Average Precision :  0.6929146073177479
Average Sensitivity :  0.6376956133013285
Average F1 Score :  0.6420665843056661
Average Specificity :  0.5440143866817081
best parameters:  {'classifier__max_depth': 10, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 10}


# Classification with SVM using Statistical Features

In [66]:
#SVM Classifier Using Statistical Features

#Train index gives indices of the training data and val_index gives indices of the validation data
#group_kf.split(features, label_array, groups=subject_array) gives pairs of arrays where each pair represents the indices of the training and testing sets for 1 fold of CV
#Ensures all samples from the same group are in the same fold(i.e samples from the same group/subject are not spread out between the folds!)
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GroupKFold, GridSearchCV



# Define the parameter grid
grid = {'classifier__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1, 3, 5, 7, 10],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'classifier__degree': [2, 3, 4],'classifier__gamma': ['scale', 'auto'],'classifier__coef0': [0.0, 0.1, 0.5] }

# Define the pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])

# Initialize GroupKFold and GridSearchCV
group_kf = GroupKFold(n_splits=5)
gscv = GridSearchCV(pipe, grid, cv=group_kf, n_jobs=16, scoring='accuracy')
for train_index, val_index in group_kf.split(features, label_array, groups=subject_array):
    # Split data
    X_train, X_val = features[train_index], features[val_index]
    y_train, y_val = label_array[train_index], label_array[val_index]

    # Fit and predict
    gscv.fit(X_train, y_train, groups=subject_array[train_index])
    y_pred = gscv.predict(X_val)

    # Calculate metrics
    accuracy_list.append(accuracy_score(y_val, y_pred))
    precision_list.append(precision_score(y_val, y_pred, average='weighted'))
    recall_list.append(recall_score(y_val, y_pred, average='weighted'))
    f1_list.append(f1_score(y_val, y_pred, average='weighted'))

    # Calculate specificity
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    specificity_list.append(tn / (tn+fp))





In [69]:
#Average accuracy, precisin, sensitivity, f1 score and specificity of model on validation set for each fold in groupkfold cv(with 5 folds on best parameters)
# Print average metrics
print('Average Accuracy : ', np.mean(accuracy_list))
print('Average Precision : ', np.mean(precision_list))
print('Average Sensitivity : ', np.mean(recall_list))
print('Average F1 Score : ', np.mean(f1_list))
print('Average Specificity : ', np.mean(specificity_list))
print('best_params: ',gscv.best_params_)

Average Accuracy :  0.6388200232249873
Average Precision :  0.6844750769734068
Average Sensitivity :  0.6388200232249873
Average F1 Score :  0.6436310304954381
Average Specificity :  0.5764648593796561
best_params:  {'classifier__C': 1, 'classifier__coef0': 0.0, 'classifier__degree': 2, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}


# Feature Extraction 2: Spectral Coherence 

In [46]:
import numpy as np
from scipy.signal import welch, csd

def compute_coherence(signal1, signal2, fs):
    # Compute the Power Spectral Densities
    freqs, psd1 = welch(signal1, fs=fs)
    _, psd2 = welch(signal2, fs=fs)

    # Compute the Cross Spectral Density
    _, csd12 = csd(signal1, signal2, fs=fs)

    # Compute the Coherence
    coherence = np.abs(csd12)**2 / (psd1 * psd2)

    return freqs, coherence

#Feature Extraction 2: Spectral Coherance
n_epochs, n_channels, _ = data_array.shape
coherence_array = []

for i in range(n_epochs):
    epoch_coherence = []
    for j in range(n_channels):
        for k in range(j+1, n_channels):
            _, coherence = compute_coherence(data_array[i, j, :], data_array[i, k, :], fs=250)
            epoch_coherence.append(coherence)
    coherence_array.append(epoch_coherence)

coherence_array = np.array(coherence_array)
#Frequencies 
freqs, _ = compute_coherence(data_array[0, 0, :], data_array[0, 1, :], fs=250)

print(freqs.shape)
print(coherence_array.shape)


(129,)
(1433, 171, 129)


In [ ]:
# Heatmap Visualization of Spectral Coherence averaged over Epochs for Healthy Controls and Schizophrenia Patients

# Classification with Logistic Regression using PCA Reduced Spectral Coherance Features

In [ ]:
#Spectral Coherance features + PCA + Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [113]:


# Reshape the coherence_array into a 2D array
X = coherence_array.reshape(coherence_array.shape[0], -1)

y = label_array
groups = subject_array
# Standardize the features to have zero mean and unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)  # Change this to the number of components you want to keep
X_pca = pca.fit_transform(X_scaled)

# Use X_pca instead of X for the rest of the code
gkf = GroupKFold(n_splits=5)

# Initialize lists to store the metrics for each fold
accuracies = []

precisions = []
sensitivities = []
f1_scores = []
specificities = []

grid = {
    'classifier__C': [0.001,0.01,0.05,0.1,0.5, 1,3,5,7,10], 
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'], 
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
pipe = Pipeline([('classifier', LogisticRegression())])
gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
gscv.fit(X_pca,y,groups=subject_array)

# Extract best parameters from gscv and remove 'classifier__' prefix
best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}
model = LogisticRegression(**best_params)
for train_index, test_index in gkf.split(X_pca, y, subject_array):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 
    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.frame

In [114]:
# Print the average metrics
print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best parameters: ',gscv.best_params_)

Average accuracy: 0.8098318853116323
Average precision: 0.8764316438424915
Average sensitivity: 0.7491648208934283
Average F1 score: 0.7941010761327485
Average specificity: 0.8232679444832935
best parameters:  {'classifier__C': 0.001, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}


# Classification with Random Forest using PCA Reduced Spectral Coherance Features

In [100]:
#Spectral Coherance features + PCA + RF 
from sklearn.ensemble import RandomForestClassifier


gkf = GroupKFold(n_splits=5)
# Initialize lists to store the metrics for each fold
accuracies = []
precisions = []
sensitivities = []
f1_scores = []
specificities = []


# Defineing grid
grid = {
    'classifier__n_estimators': [10, 50, 100, 150],
    'classifier__max_depth': [ 10, 20],
    'classifier__min_samples_split': [2, 3, 5],
    'classifier__min_samples_leaf': [1, 2, 4],
}
pipe = Pipeline([('classifier', RandomForestClassifier())])
gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
gscv.fit(X_pca,y,groups=subject_array)

# Extract performance metrics using best parameters
best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}
model = RandomForestClassifier(**best_params)
for train_index, test_index in gkf.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 
    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))

best_params: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 3, 'classifier__n_estimators': 50}


In [109]:

print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best parameters: ',gscv.best_params_)

Average accuracy: 0.5748065805391814
Average precision: 0.6070245112030573
Average sensitivity: 0.8502355064583428
Average F1 score: 0.6732213817578051
Average specificity: 0.3744024878737437
best parameters:  {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 3, 'classifier__n_estimators': 50}


# Classification with SVM using PCA Reduced Spectral Coherance Features

In [129]:
#Spectral Coherance features + PCA + SVM 
from sklearn.svm import SVC
# Use X_pca instead of X for the rest of the code
gkf = GroupKFold(n_splits=5)

# Initialize lists to store the metrics for each fold
accuracies = []
precisions = []
sensitivities = []
f1_scores = []
specificities = []

# Define the parameter grid
grid = {'classifier__C': [ 0.1, 0.5, 1, 3, 5, 7, 10],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'classifier__degree': [1,2, 3],'classifier__gamma': ['scale', 'auto'] }

pipe = Pipeline([('classifier', SVC())])


gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
# Extract best parameters from gscv and remove 'classifier__' prefix
gscv.fit(X_pca,y,groups=subject_array)
best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}

model = SVC(**best_params)

# Performance metrics using best parameters

for train_index, test_index in gkf.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 
    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))



In [130]:
# Print the average metrics
print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best parameters: ',gscv.best_params_)

Average accuracy: 0.7315923271866639
Average precision: 0.7303868054306205
Average sensitivity: 0.7672087710548103
Average F1 score: 0.7404932337862637
Average specificity: 0.7103855112944497
best parameters:  {'classifier__C': 10, 'classifier__degree': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'poly'}


# Feature Extraction 3: Phase Locked Values (PLVs)

In [118]:
#Feature Extraction 3: Phase Locked Values 
from scipy.signal import hilbert


n_epochs,n_channels,n_timepoints=data_array.shape
plv_array=[]
for i in range(n_epochs):
    epoch_plv=[]
    for j in range(n_channels):
        for k in range(j+1,n_channels):
            # Compute the analytic signals
            analytic1 = hilbert(data_array[i,j,:])
            analytic2 = hilbert(data_array[i,k,:])

            # Compute the phase difference
            phase_diff = np.angle(analytic1) - np.angle(analytic2)

            # Compute the PLV
            plv = np.abs(np.mean(np.exp(1j * phase_diff)))
            epoch_plv.append(plv)
    plv_array.append(epoch_plv)


plv_array=np.array(plv_array)
print(plv_array.shape)



(1433, 171)


In [ ]:
# Heatmap Visualization of Phase Locked Values(PLVs) averaged over Epochs for Healthy Controls and Schizophrenia Patients

# Classification with Logistic Regressioon using PCA Reduced PLVs

In [119]:


# Reshape the coherence_array into a 2D array
X = plv_array.reshape(plv_array.shape[0], -1)

y = label_array
groups = subject_array
# Standardize the features to have zero mean and unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)  # Change this to the number of components you want to keep
X_pca = pca.fit_transform(X_scaled)

# Use X_pca instead of X for the rest of the code
gkf = GroupKFold(n_splits=5)
# Initialize lists to store the metrics for each fold
accuracies = []
precisions = []
sensitivities = []
f1_scores = []
specificities = []


# Define the parameter grid
grid = {
    'classifier__C': [0.001,0.01,0.05,0.1,0.5, 1,3,5,7,10], 
    'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'], 
    'classifier__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

pipe = Pipeline([('classifier', LogisticRegression())])


gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
# Extract best parameters from gscv and remove 'classifier__' prefix
gscv.fit(X_pca,y,groups=subject_array)
best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}

model = LogisticRegression(**best_params)

for train_index, test_index in gkf.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 
    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Library/Frameworks/Python.frame

In [120]:
# Print the average metrics
print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best parameters: ',gscv.best_params_)

Average accuracy: 0.7482206182813548
Average precision: 0.7507375417419777
Average sensitivity: 0.761997987583878
Average F1 score: 0.7498097108984205
Average specificity: 0.7246282245294109
best parameters:  {'classifier__C': 0.1, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}


# Classification with Random Forest using PCA Reduced PLVs

In [122]:
# Defineing grid
grid = {
    'classifier__n_estimators': [10, 50, 100, 150],
    'classifier__max_depth': [ 10, 20],
    'classifier__min_samples_split': [2, 3, 5],
    'classifier__min_samples_leaf': [1, 2, 4],
}
pipe = Pipeline([('classifier', RandomForestClassifier())])
gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
gscv.fit(X_pca,y,groups=subject_array)

# Use X_pca instead of X for the rest of the code
gkf = GroupKFold(n_splits=5)
# Initialize lists to store the metrics for each fold
accuracies = []

precisions = []
sensitivities = []
f1_scores = []
specificities = []

best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}

model = RandomForestClassifier(**best_params)

for train_index, test_index in gkf.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 

    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))



In [123]:
# Print the average metrics
print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best_params',gscv.best_params_)

Average accuracy: 0.6280735053810387
Average precision: 0.6588396101199745
Average sensitivity: 0.7164448919255383
Average F1 score: 0.6710131878041052
Average specificity: 0.541880135839034
best_params {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 10}


# Classification with SVM using PCA Reduced PLVs

In [126]:


# Initialize lists to store the metrics for each fold
accuracies = []

precisions = []
sensitivities = []
f1_scores = []
specificities = []

# Define the parameter grid
grid = {'classifier__C': [ 0.1, 0.5, 1, 3, 5, 7, 10],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'classifier__degree': [1,2, 3],'classifier__gamma': ['scale', 'auto'] }

pipe = Pipeline([('classifier', SVC())])


gscv = GridSearchCV(pipe, grid, cv=gkf, n_jobs=16, scoring='accuracy')
# Extract best parameters from gscv and remove 'classifier__' prefix
gscv.fit(X_pca,y,groups=subject_array)
best_params = {k.replace('classifier__', ''): v for k, v in gscv.best_params_.items()}

model = SVC(**best_params)
for train_index, test_index in gkf.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy) 
    # Compute the metrics and add them to the lists
    precisions.append(precision_score(y_test, y_pred, average='binary'))
    sensitivities.append(recall_score(y_test, y_pred, average='binary'))
    f1_scores.append(f1_score(y_test, y_pred, average='binary'))

    # Compute specificity: TN / (TN + FP)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificities.append(tn / (tn + fp))



In [128]:
# Print the average metrics
print("Average accuracy:", np.mean(accuracies))
print("Average precision:", np.mean(precisions))
print("Average sensitivity:", np.mean(sensitivities))
print("Average F1 score:", np.mean(f1_scores))
print("Average specificity:", np.mean(specificities))
print('best_params: ',gscv.best_params_)

Average accuracy: 0.7315923271866639
Average precision: 0.7303868054306205
Average sensitivity: 0.7672087710548103
Average F1 score: 0.7404932337862637
Average specificity: 0.7103855112944497
best_params:  {'classifier__C': 10, 'classifier__degree': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'poly'}
